In [8]:
from bs4 import BeautifulSoup
import requests

#url = 'https://www.tekstowo.pl/artysci_na,A,strona,1.html'
# R:  pages = [i for i in range(1, 11)]
pages = list(range(1,11)) # R: we can shorten the above creation of list of ints
domain = 'https://tekstowo.pl'
urls = []
cnt = 0


# Extracting the artist list
# it is good to use more intuitive names, also the 'i' variable is also used in the loop below
for page in pages:
    url = f'https://www.tekstowo.pl/artysci_na,A,strona,{page}.html'
    try:
        req = requests.get(url)
        if req.ok:
            soup = BeautifulSoup(req.content, 'lxml')
            # R: we can use range instead of enumreate if we are not using 'i' variable
            for i, link in enumerate(soup.find_all('a')):
                # R: if type(link.get('href')) == str and 'piosenki_' in link.get('href'):
                if isinstance(link.get('href'), str) and 'piosenki_' in link.get('href'):# it is safer to use isinstance
                    urls.append(link.get('href'))

            #print(soup.prettify())
            #art = soup.find_all(class_='title')

            #print(art)

    except Exception as e:
        cnt += 1
        print(e)

len(urls)

300

In [30]:
def get_pages_number(url: str) -> int: # R: added typehints
	"""
	Get the highest page number per given letter.
	"""
	max_page = 0

	req = requests.get(url)
	if req.ok:
		soup = BeautifulSoup(req.content, 'lxml')
		for page in soup.find_all(class_='page-link'):
			# R: we can make these 2 conditions into one liner
			# R: if page.text.isnumeric():
			# 	R: if int(page.text) > max_page:
			if page.text.isnumeric() and int(page.text) > max_page:
				max_page = int(page.text)
# 			if page.text.isnumeric():
# 				if int(page.text) > max_page:
# 					max_page = int(page.text)
	#print(f"Max page: {max_page}")
	return max_page

In [31]:
import string
import time

def create_lut_pagination():
    """
    Creates a look-up table for each letter containing max page number
    """

    # R: remove below imports above the function
    # import string
    # import time
    
    # R: we can use oneline in this case (it is not as readable as the original solution)
    # R: alphabet = string.ascii_letters.upper()
    # R: alphabet = sorted(list(set(alphabet)))
    # R: alphabet.append('pozostale')
    # R: we can use oneline in this case (it is not as readable as solution above)
    
    # R: alphabet = sorted(set(string.ascii_letter.upper()))
    alphabet = list(string.ascii_uppercase) + ['pozostałe']
    
    lut_pages = {}

    for letter in alphabet:
        time.sleep(5)
        url = f"https://www.tekstowo.pl/artysci_na,{letter}.html"
        lut_pages[letter] = get_pages_number(url)
        print(f"Letter {letter} has {lut_pages[letter]} pages of artists.")
    print(type(lut_pages))
    return lut_pages

In [32]:
max_page_per_letter = create_lut_pagination()

Letter A has 538 pages of artists.
Letter B has 385 pages of artists.
Letter C has 363 pages of artists.
Letter D has 411 pages of artists.
Letter E has 226 pages of artists.
Letter F has 189 pages of artists.
Letter G has 210 pages of artists.
Letter H has 219 pages of artists.
Letter I has 130 pages of artists.
Letter J has 337 pages of artists.
Letter K has 315 pages of artists.
Letter L has 311 pages of artists.
Letter M has 534 pages of artists.
Letter N has 204 pages of artists.
Letter O has 116 pages of artists.
Letter P has 287 pages of artists.
Letter Q has 15 pages of artists.
Letter R has 272 pages of artists.
Letter S has 574 pages of artists.
Letter T has 482 pages of artists.
Letter U has 46 pages of artists.
Letter V has 118 pages of artists.
Letter W has 141 pages of artists.
Letter X has 18 pages of artists.
Letter Y has 77 pages of artists.
Letter Z has 68 pages of artists.
Letter pozostałe has 538 pages of artists.
<class 'dict'>


In [ ]:
def get_artists(letter):
	"""
	Scrape all of the artists starting with a given letter in the alphabet.
	"""
	import time
	urls = []
	limit = max_page_per_letter[letter]+1

	for page in range(1, limit):

		url = f"https://www.tekstowo.pl/artysci_na,{letter},strona,{page}.html"

		try:
			time.sleep(1)
			response = requests.get(url)
			if response.ok:
				soup = BeautifulSoup(response.content, 'lxml')
				for link in soup.find_all('a'):
					item = link.get('href')
					if type(item) == str and 'piosenki_' in item:
						urls.append('https://tekstowo.pl' + item)
		except Exception as e:
			print(e)
		print(f"Visited {page}/{limit}")
	print(f"Collected {len(urls)} artists")
	return urls

In [ ]:
get_artists("Q")

In [99]:
artist_url = "https://www.tekstowo.pl/piosenki_artysty,q_tip.html"

In [103]:
def get_artist_songs(artist_url):
	import requests
	import time

	urls = []

	next_page_available = True


	# Link construction differs, approach to be revised
	#limit = get_pages_number(artist_url)
	#print(limit)

	while next_page_available:
		time.sleep(5)		
		try:
			response = requests.get(artist_url)
			if response.ok:
				# Create BS instance
				soup = BeautifulSoup(response.content, 'html.parser')

				# Extract all songs from the webpage
				songs = soup.find_all(class_='box-przeboje')

				# Extract the artist
				artist = soup.find(class_="col-md-7 col-lg-8 px-0")
				artist = artist.text.split(" (")[0].strip()
				#print(f'Artist: {artist}')

				# Iterate through every song and collect URL
				for song in songs:
					song_title_element = song.find(class_='title')

					if song_title_element:
						if artist in song_title_element.text.strip():
							#song_title = song_title_element.text.strip()
							song_url = "https://tekstowo.pl" + song_title_element['href']
							#print(song_title, song_url)
							if not ".plpiosenka" in song_url and song_url not in urls: # dealing with invalid URLs and duplicates
								urls.append(song_url)
					else:
						pass
				
				# Check if the 'next page' button exists
				button_next_page = soup.find_all(class_='page-link')[-1]
				if 'następna' in button_next_page.text.lower():
					artist_url = "https://tekstowo.pl" + button_next_page['href']
				else:
					next_page_available = False

		except Exception as e:
			print(e)
			
	print(f"Collected {len(urls)} song URLs")
	return urls

In [104]:
get_artist_songs(artist_url)

Collected 38 song URLs


['https://tekstowo.pl/piosenka,q_tip,life_is_better___feat__norah_jones__.html',
 'https://tekstowo.pl/piosenka,q_tip,breath_and_stop.html',
 'https://tekstowo.pl/piosenka,q_tip,we_fight___we_love.html',
 'https://tekstowo.pl/piosenka,q_tip,gettin_up.html',
 'https://tekstowo.pl/piosenka,q_tip,do_it.html',
 'https://tekstowo.pl/piosenka,q_tip,end_of_time.html',
 'https://tekstowo.pl/piosenka,q_tip,all_in.html',
 'https://tekstowo.pl/piosenka,q_tip,amplified.html',
 'https://tekstowo.pl/piosenka,q_tip,believe.html',
 'https://tekstowo.pl/piosenka,q_tip,buddy.html',
 'https://tekstowo.pl/piosenka,q_tip,dance_on_glass.html',
 'https://tekstowo.pl/piosenka,q_tip,do_it__see_it__be_it.html',
 'https://tekstowo.pl/piosenka,q_tip,don_8217_t_go_breaking_my_heart__feat__demi_lovato_.html',
 'https://tekstowo.pl/piosenka,q_tip,for_the_nasty.html',
 'https://tekstowo.pl/piosenka,q_tip,get_involved.html',
 'https://tekstowo.pl/piosenka,q_tip,go_hard.html',
 'https://tekstowo.pl/piosenka,q_tip,good_

In [141]:
song_url = "https://www.tekstowo.pl/piosenka,f__kiepski__h__kiepska__m__kiepska,jestem_motylem__odcinek_337_.html"

In [163]:
def extract_song(song_url):
	song = ""
	song_translation = None

	try:
		response = requests.get(song_url)
		if response.ok:
			soup = BeautifulSoup(response.content, 'lxml')

			# Original song
			song_html = soup.find(class_='inner-text')			
			song = song_html.text.strip()

			# Translated version
			transl_html = soup.find('div', id='translation')
			song_translation = transl_html.text.strip()

	except Exception as e:
		print(e)

	return song, song_translation

In [164]:
text1, text2 = extract_song(song_url)

In [173]:
def assess_language(song_text):
	import langdetect

	if not isinstance(song_text, str):
		return False
	
	elif len(song_text) == 0:
		raise TypeError("Empty file")

	else:
		try:
			return langdetect.detect(song_text)
		except langdetect.lang_detect_exception.LangDetectException:
			return "Language detection failed"

In [174]:
assess_language(text2)

TypeError: Empty file